<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Deepfake_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Deepfake and Manipulated Media Analysis Data Download**

In [1]:
!pip install -qU yt-dlp aiohttp decord pandas pillow soundfile tqdm crawl4ai[all] nest_asyncio playwright telethon feedparser

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.2/702.2 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7

In [3]:
import os
import asyncio
import aiohttp
import json
import subprocess
from datetime import datetime
import pandas as pd
import yt_dlp
from tqdm.asyncio import tqdm
from playwright.async_api import async_playwright
import feedparser
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
# Import Crawl4AI components
from crawl4ai import AsyncWebCrawler, CrawlerRunConfig
from crawl4ai.extraction_strategy import JsonCssExtractionStrategy

#########################################
# Setup Directories and Global Variables
#########################################
BASE_DIR = os.getcwd()
SCRAPED_URLS_DIR = os.path.join(BASE_DIR, 'scraped_urls')
IMAGES_DIR      = os.path.join(BASE_DIR, 'deepfake_images')
VIDEOS_DIR      = os.path.join(BASE_DIR, 'deepfake_videos')
AUDIOS_DIR      = os.path.join(BASE_DIR, 'deepfake_audios')
TEXT_DIR        = os.path.join(BASE_DIR, 'deepfake_texts')
METADATA_FILE   = os.path.join(BASE_DIR, 'download_metadata.json')

for d in [SCRAPED_URLS_DIR, IMAGES_DIR, VIDEOS_DIR, AUDIOS_DIR, TEXT_DIR]:
    os.makedirs(d, exist_ok=True)

download_metadata = []

#########################################
# Helper: Ensure Playwright Browsers Are Installed
#########################################
def install_playwright_browsers():
    try:
        print("[SYSTEM] Installing Playwright browsers...")
        subprocess.run(["playwright", "install"], check=True)
        print("[SYSTEM] Playwright browsers installed.")
    except Exception as e:
        print(f"[SYSTEM] Error installing Playwright browsers: {e}")

In [4]:
#########################################
# 1. URL Scraping Functions using Crawl4AI and feedparser
#########################################
async def scrape_media_urls(modality, page_url, parse_function, output_filename):
    """
    Scrape media URLs from a given page URL using Crawl4AI’s AsyncWebCrawler.
    Saves URLs to CSV and returns a unique list.
    """
    print(f"[{modality.upper()}] Scraping URLs from {page_url}")
    # Define a schema that expects a key 'extracted' with a list value.
    schema = {"extracted": list}
    extraction_strategy = JsonCssExtractionStrategy(parse_function=parse_function, schema=schema)

    try:
        async with AsyncWebCrawler(extraction_strategy=extraction_strategy) as crawler:
            result = await crawler.arun(page_url, config=CrawlerRunConfig())
    except Exception as e:
        print(f"[{modality.upper()}] Error crawling {page_url}: {e}")
        result = None

    scraped_urls = []
    if result and result.extracted_content:
        try:
            data = json.loads(result.extracted_content)
            scraped_urls = data.get('extracted', [])
        except Exception as e:
            print(f"[{modality.upper()}] Error parsing extracted content: {e}")
    scraped_urls = list({url for url in scraped_urls if url})
    df = pd.DataFrame({'url': scraped_urls})
    df.to_csv(output_filename, index=False)
    print(f"[{modality.upper()}] Saved {len(scraped_urls)} URLs to {output_filename}")
    return scraped_urls

def parse_image_links(response):
    return response.css("img::attr(src)").getall()

def parse_video_links(response):
    return response.css("a::attr(href)").getall() or response.css("video::attr(src)").getall()

def parse_audio_links(response):
    return response.css("audio::attr(src)").getall()

def parse_generic_links(response):
    return response.css("a::attr(href)").getall()

async def scrape_rss_feed_urls(modality, page_url, output_filename):
    print(f"[{modality.upper()}] Scraping RSS feed from {page_url}")
    try:
        async with aiohttp.ClientSession() as session:
            async with session.get(page_url) as resp:
                feed_text = await resp.text()
        feed = feedparser.parse(feed_text)
        rss_urls = []
        for entry in feed.entries:
            if ('deepfake' in entry.title.lower()) or ('deepfake' in entry.get('summary', '').lower()):
                rss_urls.append(entry.link)
        rss_urls = list(set(rss_urls))
        df = pd.DataFrame({'url': rss_urls})
        df.to_csv(output_filename, index=False)
        print(f"[{modality.upper()}] Saved {len(rss_urls)} RSS feed URLs to {output_filename}")
        return rss_urls
    except Exception as e:
        print(f"[{modality.upper()}] Error scraping RSS feed from {page_url}: {e}")
        return []

In [5]:
#########################################
# 2. Functions to Load URLs (with Fallbacks)
#########################################
def load_scraped_urls(modality):
    filepath = os.path.join(SCRAPED_URLS_DIR, f"{modality}_urls.csv")
    if os.path.exists(filepath):
        df = pd.read_csv(filepath)
        urls = df['url'].dropna().tolist()
        print(f"[{modality.upper()}] Loaded {len(urls)} URLs from {filepath}")
        return urls
    else:
        print(f"[{modality.upper()}] File {filepath} not found. Using fallback method.")
        return []

def fallback_image_urls(n=20, seed_keyword="deepfake"):
    return [f"https://picsum.photos/seed/{seed_keyword}{i}/600/400" for i in range(n)]

def fallback_video_urls(n=20):
    return []  # Let yt-dlp fallback handle video search if needed

def fallback_audio_urls(n=20):
    return []  # Return an empty list; our extraction fallback will be used

def fallback_text_urls(n=20):
    sample_articles = [
        "https://www.theguardian.com/technology/2024/oct/16/its-not-me-its-just-my-face-the-models-who-found-their-likenesses-had-been-used-in-ai-propaganda",
        "https://www.wired.com/story/generative-ai-deepfakes/",
        "https://www.ft.com/content/7f22ce59-1c6c-4d84-bca8-dc539992e286",
        "https://nypost.com/2024/07/17/lifestyle/how-people-are-being-tricked-by-deepfake-doctor-videos/",
        "https://www.openfox.com/deepfakes-and-their-impact-on-society/"
    ]
    return (sample_articles * ((n // len(sample_articles)) + 1))[:n]

In [6]:
#########################################
# 3. Asynchronous Download Functions with Retry Logic
#########################################
async def download_file(session, url, filename, modality, retries=3):
    for attempt in range(1, retries + 1):
        try:
            async with session.get(url) as resp:
                if resp.status == 200:
                    content = await resp.read()
                    with open(filename, "wb") as f:
                        f.write(content)
                    print(f"[{modality.upper()}] Downloaded: {filename}")
                    download_metadata.append({
                        "modality": modality,
                        "filename": filename,
                        "url": url,
                        "download_time": datetime.now().isoformat()
                    })
                    return
                else:
                    print(f"[{modality.upper()}] Attempt {attempt}: Status {resp.status} for {url}")
        except Exception as e:
            print(f"[{modality.upper()}] Attempt {attempt}: Exception for {url}: {e}")
        await asyncio.sleep(1)
    print(f"[{modality.upper()}] Failed to download {url} after {retries} attempts.")

async def download_images(urls):
    print("[IMAGE] Starting image downloads...")
    async with aiohttp.ClientSession() as session:
        tasks = []
        for idx, url in enumerate(urls):
            filename = os.path.join(IMAGES_DIR, f"deepfake_image_{idx+1:03d}.jpg")
            tasks.append(download_file(session, url, filename, modality="image"))
        await asyncio.gather(*tasks)

async def download_audios(urls):
    if not urls:
        print("[AUDIO] No audio URLs provided for download.")
        return
    print("[AUDIO] Starting audio downloads...")
    async with aiohttp.ClientSession() as session:
        tasks = []
        for idx, url in enumerate(urls):
            filename = os.path.join(AUDIOS_DIR, f"deepfake_audio_{idx+1:03d}.mp3")
            tasks.append(download_file(session, url, filename, modality="audio"))
        await asyncio.gather(*tasks)

async def download_texts(urls):
    if not urls:
        print("[TEXT] No text URLs provided for download.")
        return
    print("[TEXT] Starting text downloads...")
    async with aiohttp.ClientSession() as session:
        tasks = []
        for idx, url in enumerate(urls):
            filename = os.path.join(TEXT_DIR, f"deepfake_text_{idx+1:03d}.html")
            tasks.append(download_file(session, url, filename, modality="text"))
        await asyncio.gather(*tasks)

def download_videos(video_urls, num_videos=20):
    if video_urls:
        download_list = video_urls[:num_videos]
        print("[VIDEO] Downloading videos from scraped URLs...")
        for url in download_list:
            ydl_opts = {
                'format': 'bestvideo+bestaudio/best',
                'outtmpl': os.path.join(VIDEOS_DIR, '%(id)s.%(ext)s'),
                'merge_output_format': 'mp4',
                'quiet': True,
                'no_warnings': True,
            }
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                try:
                    ydl.download([url])
                    download_metadata.append({
                        "modality": "video",
                        "filename": url,
                        "url": url,
                        "download_time": datetime.now().isoformat()
                    })
                    print(f"[VIDEO] Downloaded: {url}")
                except Exception as e:
                    print(f"[VIDEO] Error downloading {url}: {e}")
    else:
        search_query = "ytsearch20:deepfake compilation"
        ydl_opts = {
            'format': 'bestvideo+bestaudio/best',
            'outtmpl': os.path.join(VIDEOS_DIR, '%(id)s.%(ext)s'),
            'merge_output_format': 'mp4',
            'quiet': True,
            'no_warnings': True,
        }
        print("[VIDEO] No scraped video URLs; using yt-dlp search fallback...")
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            try:
                ydl.download([search_query])
                print("[VIDEO] Fallback video downloads complete.")
            except Exception as e:
                print(f"[VIDEO] yt-dlp search fallback error: {e}")

In [7]:
#########################################
# 4. Audio Extraction via Playwright (Fallback)
#########################################
async def extract_audio_links():
    print("[AUDIO] Extracting audio URLs via Playwright fallback...")
    extracted_links = []
    try:
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            context = await browser.new_context()
            page = await context.new_page()
            # Fallback extraction from a sample audio deepfake exploration page
            url = "https://uberduck.ai/explore"
            await page.goto(url)
            await page.wait_for_timeout(5000)
            audio_elements = await page.query_selector_all("audio")
            for el in audio_elements:
                src = await el.get_attribute("src")
                if src:
                    extracted_links.append(src)
            await browser.close()
            print(f"[AUDIO] Extracted {len(extracted_links)} audio URLs via Playwright.")
    except Exception as e:
        if "Executable doesn't exist" in str(e):
            install_playwright_browsers()
        print(f"[AUDIO] Playwright extraction error: {e}")
    return extracted_links

In [8]:
#########################################
# 5. Save Download Metadata to JSON
#########################################
def save_download_metadata(metadata_list, filename=METADATA_FILE):
    try:
        with open(filename, 'w') as f:
            json.dump(metadata_list, f, indent=4)
        print(f"[SYSTEM] Download metadata saved to {filename}")
    except Exception as e:
        print(f"[SYSTEM] Error saving metadata: {e}")

In [9]:
#########################################
# 6. Main Pipeline with Expanded Sources
#########################################
async def main_pipeline():
    # Define expanded source lists.

    # Video sources: (Dataset name, URL)
    video_sources = [
        ("UADFV", "https://ai.facebook.com/datasets/dfdc/"),
        ("Deepfake-TIMIT", "https://github.com/ondyari/FaceForensics"),
        ("DFFD", "https://github.com/EndlessSora/DeeperForensics-1.0"),
        ("Celeb-DF", "https://github.com/cypw/Celeb-DF"),
        ("DFDC", "https://ai.facebook.com/datasets/dfdc/"),
        ("FaceForensics++", "https://github.com/ondyari/FaceForensics"),
        ("FFIW-10K", "https://github.com/ondyari/FaceForensics"),
        ("DeeperForensics-1.0", "https://github.com/EndlessSora/DeeperForensics-1.0"),
        ("WildDeepfake", "https://github.com/OpenTAI/wild-deepfake"),
        ("ForgeryNet", "https://github.com/duxingdong/ForgeryNet"),
        ("AV-Deepfake1M", "https://github.com/ControlNet/AV-Deepfake1M"),
        ("DeepFake MNIST+", "https://github.com/yourplaceholder/DeepFakeMNISTplus")
    ]

    # Image sources: (Dataset name, URL)
    image_sources = [
        ("DFFD_Image", "https://github.com/ondyari/FaceForensics"),
        ("iFakeFaceDB", "https://github.com/YourRepository/iFakeFaceDB"),  # Replace with actual URL if available.
        ("100kFaces", "https://thiswaifudoesnotexist.net/"),
        ("FFHQ", "https://github.com/NVlabs/ffhq-dataset"),
        ("ForgeryNet_Image", "https://github.com/duxingdong/ForgeryNet"),
        ("DeepFakeRealImages", "https://www.kaggle.com/datasets/manjilkarki/deepfake-and-real-images")
    ]

    # Audio sources: (Dataset name, URL)
    audio_sources = [
        ("WaveFake", "https://paperswithcode.com/dataset/wavefake"),
        ("DEEP-VOICE", "https://www.kaggle.com/datasets/birdy654/deep-voice-deepfake-voice-recognition"),
        ("InTheWild_Audio", "https://www.kaggle.com/datasets/abdallamohamed312/in-the-wild-audio-deepfake"),
        ("Codecfake", "https://arxiv.org/abs/2405.04880"),
        ("CrossDomain_ADD", "https://arxiv.org/abs/2404.04904"),
        ("DeepFake-Audio-Rangers_Arabic", "https://huggingface.co/datasets/DeepFake-Audio-Rangers/Arabic_Audio_Deepfake"),
        ("SONICS", "https://paperswithcode.com/dataset/sonics")
    ]

    # Process video sources.
    for name, url in video_sources:
        print(f"\n[VIDEO SOURCE: {name}]")
        output_file = os.path.join(SCRAPED_URLS_DIR, f"video_{name}_urls.csv")
        urls = await scrape_media_urls(name, url, parse_video_links, output_file)
        if urls:
            urls = urls[:20]
        else:
            print(f"[{name.upper()}] No URLs scraped; using fallback.")
            urls = fallback_video_urls(20)
        download_videos(urls, num_videos=20)

    # Process image sources.
    for name, url in image_sources:
        print(f"\n[IMAGE SOURCE: {name}]")
        output_file = os.path.join(SCRAPED_URLS_DIR, f"image_{name}_urls.csv")
        urls = await scrape_media_urls(name, url, parse_image_links, output_file)
        if urls:
            urls = urls[:20]
        else:
            print(f"[{name.upper()}] No URLs scraped; using fallback.")
            urls = fallback_image_urls(20, seed_keyword=name)
        await download_images(urls)

    # Process audio sources.
    all_audio_urls = []
    for name, url in audio_sources:
        print(f"\n[AUDIO SOURCE: {name}]")
        output_file = os.path.join(SCRAPED_URLS_DIR, f"audio_{name}_urls.csv")
        urls = await scrape_media_urls(name, url, parse_audio_links, output_file)
        if urls:
            print(f"[{name.upper()}] Scraped {len(urls)} audio URLs.")
            all_audio_urls.extend(urls)
        else:
            print(f"[{name.upper()}] No URLs scraped from page {url}.")
    # Remove duplicates.
    all_audio_urls = list({u for u in all_audio_urls if u})
    # If no URLs collected from pages, use fallback extraction.
    if not all_audio_urls:
        print("[AUDIO] No audio URLs scraped from all sources; using Playwright extraction fallback.")
        all_audio_urls = await extract_audio_links()
    # Limit to 50 audio URLs.
    all_audio_urls = all_audio_urls[:50]
    await download_audios(all_audio_urls)

    # Optionally, process text via an RSS feed.
    rss_url = "https://www.wired.com/feed/category/tech/latest/rss"
    rss_output = os.path.join(SCRAPED_URLS_DIR, "deepfake_news_rss_urls.csv")
    rss_urls = await scrape_rss_feed_urls("deepfake_news_rss", rss_url, rss_output)
    if rss_urls:
        rss_urls = rss_urls[:20]
        await download_texts(rss_urls)
    else:
        print("[TEXT] No RSS feed URLs scraped; using fallback.")
        await download_texts(fallback_text_urls(20))

    # Save all download metadata.
    save_download_metadata(download_metadata)
    print("[SYSTEM] All downloads complete. Check the respective directories for files.")

In [10]:
#########################################
# 7. Entry Point with Event Loop Handling
#########################################
import nest_asyncio
nest_asyncio.apply()

try:
    loop = asyncio.get_running_loop()
except RuntimeError:
    loop = None

if loop and loop.is_running():
    print("[SYSTEM] Detected running event loop. Using loop.run_until_complete()...")
    loop.run_until_complete(main_pipeline())
else:
    asyncio.run(main_pipeline())

[SYSTEM] Detected running event loop. Using loop.run_until_complete()...

[VIDEO SOURCE: UADFV]
[UADFV] Scraping URLs from https://ai.facebook.com/datasets/dfdc/
[UADFV] Error crawling https://ai.facebook.com/datasets/dfdc/: BrowserType.launch: Executable doesn't exist at /root/.cache/ms-playwright/chromium-1155/chrome-linux/chrome
╔════════════════════════════════════════════════════════════╗
║ Looks like Playwright was just installed or updated.       ║
║ Please run the following command to download new browsers: ║
║                                                            ║
║     playwright install                                     ║
║                                                            ║
║ <3 Playwright Team                                         ║
╚════════════════════════════════════════════════════════════╝
[UADFV] Saved 0 URLs to /content/scraped_urls/video_UADFV_urls.csv
[UADFV] No URLs scraped; using fallback.
[VIDEO] No scraped video URLs; using yt-dlp search fal

ERROR: unable to download video data: HTTP Error 403: Forbidden


[VIDEO] yt-dlp search fallback error: ERROR: unable to download video data: HTTP Error 403: Forbidden

[VIDEO SOURCE: Deepfake-TIMIT]
[DEEPFAKE-TIMIT] Scraping URLs from https://github.com/ondyari/FaceForensics
[DEEPFAKE-TIMIT] Error crawling https://github.com/ondyari/FaceForensics: BrowserType.launch: Executable doesn't exist at /root/.cache/ms-playwright/chromium-1155/chrome-linux/chrome
╔════════════════════════════════════════════════════════════╗
║ Looks like Playwright was just installed or updated.       ║
║ Please run the following command to download new browsers: ║
║                                                            ║
║     playwright install                                     ║
║                                                            ║
║ <3 Playwright Team                                         ║
╚════════════════════════════════════════════════════════════╝
[DEEPFAKE-TIMIT] Saved 0 URLs to /content/scraped_urls/video_Deepfake-TIMIT_urls.csv
[DEEPFAKE-TIMIT] 

ERROR: unable to download video data: HTTP Error 403: Forbidden


[VIDEO] yt-dlp search fallback error: ERROR: unable to download video data: HTTP Error 403: Forbidden

[VIDEO SOURCE: DFFD]
[DFFD] Scraping URLs from https://github.com/EndlessSora/DeeperForensics-1.0
[DFFD] Error crawling https://github.com/EndlessSora/DeeperForensics-1.0: BrowserType.launch: Executable doesn't exist at /root/.cache/ms-playwright/chromium-1155/chrome-linux/chrome
╔════════════════════════════════════════════════════════════╗
║ Looks like Playwright was just installed or updated.       ║
║ Please run the following command to download new browsers: ║
║                                                            ║
║     playwright install                                     ║
║                                                            ║
║ <3 Playwright Team                                         ║
╚════════════════════════════════════════════════════════════╝
[DFFD] Saved 0 URLs to /content/scraped_urls/video_DFFD_urls.csv
[DFFD] No URLs scraped; using fallback.
[VIDEO]

...

[VIDEO] yt-dlp search fallback error: ERROR: unable to download video data: HTTP Error 403: Forbidden

[VIDEO SOURCE: AV-Deepfake1M]
[AV-DEEPFAKE1M] Scraping URLs from https://github.com/ControlNet/AV-Deepfake1M
[AV-DEEPFAKE1M] Error crawling https://github.com/ControlNet/AV-Deepfake1M: BrowserType.launch: Executable doesn't exist at /root/.cache/ms-playwright/chromium-1155/chrome-linux/chrome
╔════════════════════════════════════════════════════════════╗
║ Looks like Playwright was just installed or updated.       ║
║ Please run the following command to download new browsers: ║
║                                                            ║
║     playwright install                                     ║
║                                                            ║
║ <3 Playwright Team                                         ║
╚════════════════════════════════════════════════════════════╝
[AV-DEEPFAKE1M] Saved 0 URLs to /content/scraped_urls/video_AV-Deepfake1M_urls.csv
[AV-DEEPFAKE1M] 

ERROR: unable to download video data: HTTP Error 403: Forbidden


[VIDEO] yt-dlp search fallback error: ERROR: unable to download video data: HTTP Error 403: Forbidden

[VIDEO SOURCE: DeepFake MNIST+]
[DEEPFAKE MNIST+] Scraping URLs from https://github.com/yourplaceholder/DeepFakeMNISTplus
[DEEPFAKE MNIST+] Error crawling https://github.com/yourplaceholder/DeepFakeMNISTplus: BrowserType.launch: Executable doesn't exist at /root/.cache/ms-playwright/chromium-1155/chrome-linux/chrome
╔════════════════════════════════════════════════════════════╗
║ Looks like Playwright was just installed or updated.       ║
║ Please run the following command to download new browsers: ║
║                                                            ║
║     playwright install                                     ║
║                                                            ║
║ <3 Playwright Team                                         ║
╚════════════════════════════════════════════════════════════╝
[DEEPFAKE MNIST+] Saved 0 URLs to /content/scraped_urls/video_DeepFake MNIS

ERROR: unable to download video data: HTTP Error 403: Forbidden


[VIDEO] yt-dlp search fallback error: ERROR: unable to download video data: HTTP Error 403: Forbidden

[IMAGE SOURCE: DFFD_Image]
[DFFD_IMAGE] Scraping URLs from https://github.com/ondyari/FaceForensics
[DFFD_IMAGE] Error crawling https://github.com/ondyari/FaceForensics: BrowserType.launch: Executable doesn't exist at /root/.cache/ms-playwright/chromium-1155/chrome-linux/chrome
╔════════════════════════════════════════════════════════════╗
║ Looks like Playwright was just installed or updated.       ║
║ Please run the following command to download new browsers: ║
║                                                            ║
║     playwright install                                     ║
║                                                            ║
║ <3 Playwright Team                                         ║
╚════════════════════════════════════════════════════════════╝
[DFFD_IMAGE] Saved 0 URLs to /content/scraped_urls/image_DFFD_Image_urls.csv
[DFFD_IMAGE] No URLs scraped; using f